In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# keep only docsuments with covid -cov-2 and cov2
def search_focus(df):
    dfa = df[df['abstract'].str.contains('covid')]
    dfb = df[df['abstract'].str.contains('-cov-2')]
    dfc = df[df['abstract'].str.contains('cov2')]
    dfd = df[df['abstract'].str.contains('ncov')]
    frames=[dfa,dfb,dfc,dfd]
    df = pd.concat(frames)
    df=df.drop_duplicates(subset='title', keep="first")
    return df

# load the meta data from the CSV file using 3 columns (abstract, title, authors),
df=pd.read_csv('/kaggle/input/CORD-19-research-challenge/metadata.csv', usecols=['title','journal','abstract','authors','doi','publish_time','sha'])
print (df.shape)
#drop duplicates
#df=df.drop_duplicates()
#drop NANs 
df=df.fillna('no data provided')
df = df.drop_duplicates(subset='title', keep="first")
df=df[df['publish_time'].str.contains('2020')]
# convert abstracts to lowercase
df["abstract"] = df["abstract"].str.lower()+df["title"].str.lower()
#show 5 lines of the new dataframe
df=search_focus(df)
print (df.shape)
df.head()

In [ ]:
#!/usr/bin/python3 
!pip install transformers 
!pip install SentencePiece    
#If installing the above restart the kernel for it to work.                                                                                             
from transformers import T5Tokenizer, T5ForConditionalGeneration
tasks = ["How long are individuals are contagious?"]

def initialize():
  global tokenizer, model
  tokenizer = T5Tokenizer.from_pretrained('allenai/unifiedqa-t5-small')#t5-3b not t5-3B
  model = T5ForConditionalGeneration.from_pretrained('allenai/unifiedqa-t5-small', return_dict=True)

# You can also use  and "
def task_and_reply(task):
  input_ids = tokenizer(task, return_tensors="pt").input_ids 
  outputs = model.generate(input_ids)
  decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return decoded

#takes the question, abstract, and full sentence answer and applies the task of finding the uncommon part in the full sentence answer
#does this for every abstract
def find_answer(abstract):
    #this function finds the answer in an abstract provided
    GA_Tech_Question_1 = "How long are individuals contagious?"
    task = "question: %s context:%s" %(GA_Tech_Question_1, abstract)
    result = task_and_reply(task)
    return result

initialize()
find_answer("Individuals are contagious for 10 days long.")

In [ ]:
#for abstract in df['abstract']:
    #answer = find_answer(abstract)
    #print(answer)
    
#finds the true and false answers 
    
def entailment(answer):
    #this function finds the answer in an abstract provided
    GA_Tech_Question_1 = "How long are individuals contagious?"
    task = "qnli question: %s sentence:%s" %(GA_Tech_Question_1, answer)
    result = task_and_reply(task)
    if result == "entailment": return True
    else: return False

In [ ]:
#uses keywords to deliver more accurate results
#at least one of the keywords must be in whole abstract
phrases = ['shedding duration','contagious period','duration of shedding', 'viral']
i = 0
results = list()
row = list()
for abstract in df['abstract']:
    for phrase in phrases:
        if phrase in abstract:
            answer = find_answer(abstract)
            entailing = entailment(answer)
            if entailing:
                results.append(answer)
                row.append(i)
                print(answer)
    if i % 20 == 0:
        print("I just completed question #:", i)
    if i == 2000: break
    i = i + 1
    
print(results)
print(row)

In [ ]:
#breaks abstract into separate sentences, and if the answer excerpt is in the sentence, it will print the whole sentence 
results2 = [" "]*len(results)
for r in row:
    #for row in rows
    sentences = list(df['abstract'])[r]
    sentences = sentences.split('. ')
    answer = results[row.index(r)]
    for sentence in sentences:
        if answer in sentence:
            results2[row.index(r)] = sentence

        
for result in results2:
        print(result)
        print()

In [ ]:
#results contain all the answers from entailing and keywording
#results2 contains results, but the answer with its sentence 

import string
string.ascii_uppercase

#puts the letter choice next to the answer

def joinanswers(answers):
    joinedanswers = " "
    for ltr, answer in zip(string.ascii_uppercase, answers):
        joinedanswers = joinedanswers + "(%s) %s " % (ltr, answer)
    return joinedanswers

def multichoice(answers):
    joinedanswers = joinanswers(answers)
    GA_Tech_Question_1 = "How long are individuals contagious?"
    task = "question: %s answers: %s" % (GA_Tech_Question_1, joinedanswers)
    result = task_and_reply(task)
    return result



In [ ]:
results3 = []
n_results = len(results)
for i in range(0,n_results,4):
    choices_4 = results2[i:i+4]
    result = multichoice(choices_4)
    results3.append(result)
print(results3)